In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../..")

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from librep.datasets.har.loaders import (
    MotionSense_BalancedView20HZ
)

2022-11-10 20:07:22.773822: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 20:07:22.773842: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
from librep.transforms.topo_ae import (
    TopologicalDimensionalityReduction,
    ConvolutionalTopologicalDimensionalityReduction,
    CustomTopoDimRedTransform
)
from librep.transforms import UMAP
from sklearn.manifold import TSNE
from experiments.Topological_ae.Experiment_utils import *
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [6]:
# MotionSense Loader
loader = MotionSense_BalancedView20HZ(
    root_dir="../../../data/views/MotionSense/balanced_view_20Hz_with_gravity_9.81_acc_standard", 
    download=False
)

# Print the readme (optional)
loader.print_readme()

# Balanced MotionSense View Resampled to 20Hz with Gravity - Multiplied acc by 9.81m/s²

This is a view from [MotionSense] that was spllited into 3s windows and was resampled to 20Hz using the [FFT method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html#scipy.signal.resample). 

The data was first splitted in three sets: train, validation and test. Each one with the following proportions:
- Train: 70% of samples
- Validation: 10% of samples
- Test: 20% of samples

After splits, the datasets were balanced in relation to the activity code column, that is, each subset have the same number of activitiy samples.

**NOTE**: Each subset contain samples from distinct users, that is, samples of one user belongs exclusivelly to one of three subsets.

## Activity codes
- 0: downstairs (569 train, 101 validation, 170 test) 
- 1: upstairs (569 train, 101 validation, 170 test) 
- 2: sitting (569 train, 101 validation, 170 test) 
- 3: standing (569 train, 101 validation, 170 test) 
- 4: walking (569 train, 101 validation, 170 test) 
- 5: jogging (569 train, 101 validation, 170 test) 
 

## Standartized activity codes
- 0: sit (569 train, 101 validation, 170 test) 
- 1: stand (569 train, 101 validation, 170 test) 
- 2: walk (569 train, 101 validation, 170 test) 
- 3: stair up (569 train, 101 validation, 170 test) 
- 4: stair down (569 train, 101 validation, 170 test) 
- 5: run (569 train, 101 validation, 170 test) 
      




In [7]:
# Load the dataset
# If concat_train_validation is true, return a tuple (train+validation, test)
train_val, test = loader.load(concat_train_validation=True, label='user')
train_val, test

(PandasMultiModalDataset: samples=4020, features=360, no. window=6, label_columns='user',
 PandasMultiModalDataset: samples=1020, features=360, no. window=6, label_columns='user')

In [8]:
train_HD = np.array(train_val[:][0])
train_Y = np.array(train_val[:][1])
test_HD = np.array(test[:][0])
test_Y = np.array(test[:][1])

In [9]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 360) (4020,) (1020, 360) (1020,)


In [10]:
# Como hacer un dataset válido?
# Como fue extraido kuhar?

y_df = pd.DataFrame(test_Y)
# np.unique(test_Y)
for val in y_df.groupby(0):
    # print('\n', val[0], val[1])
    print(val)

(3,       0
5     3
12    3
15    3
21    3
22    3
...  ..
1005  3
1008  3
1010  3
1015  3
1018  3

[222 rows x 1 columns])
(14,        0
0     14
18    14
19    14
44    14
47    14
...   ..
987   14
992   14
994   14
1011  14
1012  14

[183 rows x 1 columns])
(18,        0
4     18
6     18
7     18
8     18
11    18
...   ..
1007  18
1013  18
1014  18
1017  18
1019  18

[223 rows x 1 columns])
(19,        0
1     19
2     19
3     19
9     19
10    19
...   ..
998   19
999   19
1001  19
1006  19
1009  19

[233 rows x 1 columns])
(24,        0
16    24
24    24
29    24
32    24
33    24
...   ..
974   24
977   24
979   24
1002  24
1016  24

[159 rows x 1 columns])


In [20]:
print(np.unique(train_Y))
print(np.unique(test_Y))

[ 1  2  4  5  6  7  8  9 10 11 12 13 15 16 17 20 21 22 23]
[ 3 14 18 19 24]


# Aplicar FFT

In [12]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(
    transforms=[fft_transform],
    new_window_name_prefix="fft."
)

In [13]:
train_dataset_fft = transformer(train_val)
# validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test)

In [14]:
print(train_dataset_fft.X.shape)
# print(validation_dataset_fft.X.shape)
print(test_dataset_fft.X.shape)

(4020, 180)
(1020, 180)


In [15]:
print(np.max(train_dataset_fft.X))

797.3368276806895


In [16]:
train_dataset_fft.X

array([[117.47446671,   7.69616708,   9.12345317, ...,   1.58565333,
          1.92709353,   2.56103481],
       [ 56.99305528,   7.03810632,  21.13174295, ...,   3.59397955,
          1.24491107,   2.19807336],
       [ 43.67148129,   9.31303897,  48.54955013, ...,   0.20668447,
          2.1782541 ,   1.17453151],
       ...,
       [ 48.49117037,   5.91621941,  13.46303591, ...,   0.95960646,
          3.65355504,   4.60541355],
       [ 88.69039888,  27.8862055 ,  19.11225658, ...,   1.48263297,
          3.9119881 ,   2.55326373],
       [ 55.520648  ,   2.58290685,   8.95693627, ...,   5.4340967 ,
          4.2408613 ,  10.63953696]])

In [17]:
train_HD = train_dataset_fft.X
train_LD = None
train_Y = train_dataset_fft.y
test_HD = test_dataset_fft.X
test_LD = None
test_Y = test_dataset_fft.y

In [18]:
print(train_HD.shape, train_Y.shape, test_HD.shape, test_Y.shape)

(4020, 180) (4020,) (1020, 180) (1020,)


In [19]:
asd

NameError: name 'asd' is not defined